# connie-skipper

Repository at [https://github.com/PhMota/connie-skipper](https://github.com/PhMota/connie-skipper)

In [1]:
%load_ext autoreload
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
# import gc
from glob import glob
import xarray as xr
import connie_skipper as sk
import warnings
warnings.simplefilter("ignore", category=xr.core.extensions.AccessorRegistrationWarning)

In [2]:
from IPython.display import display, IFrame, clear_output
from ipywidgets import interact, interactive_output
import ipywidgets as widgets

In [3]:
class App:
    def __init__(self, rootpath):
        self.output = widgets.Output()
        self.select = widgets.Select( 
            options = [ (file.split(rootpath)[-1], file) for file in sorted(glob(rootpath+"*/*.fz"))[::-1] ], 
            layout = {'width': '90%'},
            rows = 10
        )
        self.progress = widgets.IntProgress(
            value = 0, 
            min = 0, 
            max = 10, 
            description = 'loading'
        )
        self.app = None
        with self.output:
            display(self.select, self.progress)
        display(self.output)
        self.select.observe( 
            self.plot_from_path2,
            names = "value"
        )
        self.interactive_figure = None
        
    def plot_from_path2(self, path):
        self.progress.value = 1
        print( path.new )
        self.da = sk.fits_to_DataArray( path.new )

        self.progress.value = 5
        chid_max = len(self.da["chid"].data)-1
        samp_max = int(self.da["samp"].data[-1])
        app_dict = {}
        if self.app is None:
            trim = widgets.Checkbox(
                description = 'trim', 
                value = True
            )
            rowproj = widgets.Checkbox(description='rowproj')
            colproj = widgets.Checkbox(description='colproj')
            chid = widgets.IntSlider(
                value = 0,
                min = 0,
                max = chid_max,
                description = f'chid[0─{chid_max}]'
            )
            samp = widgets.IntSlider( 
                value = 0,
                min = 0, 
                max = samp_max,
                description = f'samp[0─{samp_max}]'
            )
            stats = widgets.Dropdown( options=["none", "median", "mean", "std", "mad", "min", "max"], description='stats' )
            statsRange = widgets.IntRangeSlider(
                value = [0, samp_max+1],
                min = 0,
                max = samp_max+1,
                description = f"{stats.value}"
            )
            stats.observe( lambda opt: setattr(statsRange, "description", opt.new), names="value" )
            ui = widgets.VBox([ trim, rowproj, colproj, chid, samp, stats, statsRange])
            app_dict = {
                "rowproj": rowproj, 
                "colproj": colproj, 
                "trim": trim, 
                "chid": chid, 
                "samp": samp, 
                "stats": stats, 
                "stats_range": statsRange
            } 
        else:
            self.app["chid"].max = chid_max
            self.app["chid"].description = f'chid[0─{chid_max}]'
            self.app["samp"].max = samp_max
            self.app["samp"].description = f'samp[0─{samp_max}]'
            self.app["stats_range"].value = [0, samp_max+1]
            self.app["stats_range"].max = samp_max+1
            app_dict = self.app

        self.progress.value = 7
        plot = interactive_output( 
            self.gen_plot, 
            app_dict
        )
        self.progress.value = 7
        if self.app is None:
            with self.output:
                display( widgets.HBox([ui,plot]) )
            self.app = app_dict
        self.progress.value = 10        
        
    def gen_plot(self, 
        rowproj, 
        colproj, 
        trim, 
        chid, 
        samp, 
        stats, 
        stats_range
    ):
        if self.interactive_figure is not None:
            self.interactive_figure.clf()
        
        op = lambda _da: _da.isel(chid=chid)
        if stats == "none":
            stats_str = stats
            op = lambda _da, prevop=op: prevop(_da).isel(samp=samp)
        else:
            stats_str = f"{stats}({stats_range[0]}, {stats_range[1]})"
            op = lambda _da, prevop=op: prevop(_da).isel(samp=slice(*stats_range)).skipper.stats(dim = "samp", mode = stats)
        if trim:
            op = lambda _da, prevop=op: prevop(_da).skipper.trim()
        self.interactive_figure = op(self.da).skipper.plot_full( 
            robust = True, 
            fig = self.interactive_figure, 
            title = "test",
            xproj = rowproj,
            yproj = colproj,
            suptitle = f"chid={chid} samp={samp} stats={stats_str}"
        )

App("/share/storage2/connie/data/lta/testAngra/");

Output()